In [69]:
import numpy as np
import sympy as sp
import pandas as pd

In [70]:
epsilon = 1e-13

In [71]:
def LU(M):
    L = np.zeros((len(M), len(M[0])), dtype=np.float64)
    U = np.zeros((len(M), len(M[0])), dtype=np.float64)
    for i in range(len(L)):
        L[i][i] = 1
    
    for i in range(0, len(M)):
        for j in range(0, len(M[0])):
            if i <= j:
                comb1 = 0
                for k in range(0, i):
                    comb1 += L[i][k] * U[k][j]
                U[i][j] = M[i][j] - comb1
            else:
                comb1 = 0
                for k in range(0, j):
                    comb1 += L[i][k] * U[k][j]
                L[i][j] = (M[i][j] - comb1) / U[j][j]
    return L, U


def get_y(L, b):
    y1 = []
    for i in range(0, len(b)):
        comb2 = 0
        for k in range(0, i):
            comb2 += L[i][k] * y1[k]
        y1.append(b[i] - comb2)
    return np.array(y1)


def get_x(U, y1):
    x1 = np.zeros(len(y1), dtype=np.float64)
    for i in range(len(y1)-1, -1, -1):
        comb3 = 0
        for k in range(len(y1)-1, i, -1):
            comb3 += U[i][k] * x1[k]
        x1[i] = (y1[i] - comb3) / U[i][i]
    return x1


def LU_method(Matrix, B_matrix):
    L1, U1 = LU(Matrix)
    y = get_y(L1, B_matrix)
    x = get_x(U1, y)
    return sp.Matrix(x)


def inversed_iteration_method(Matrix, x_k, epsilon):
    alpha_k = max(abs(x_k))
    lambda_k = 1/alpha_k
    k = 0
    df = pd.DataFrame(np.array([]))
    while True:
        x_k_1 = LU_method(np.array(Matrix), np.array(x_k/alpha_k))
        alpha_k_1 = max(abs(x_k_1))
        
        difference = abs(lambda_k - 1/alpha_k_1)
        
        df = pd.concat([df,\
        pd.DataFrame(np.concatenate([np.array(x_k_1.T), np.matrix(1/alpha_k_1), np.matrix(difference)], axis=1))], axis = 0,\
                      ignore_index = True)
        
        if (difference < epsilon):
            df.columns = [f'X_{i+1}' if i < len(Matrix) \
                          else 'lambda' if i == len(Matrix) 
                          else '|1/alpha_(k+1) - 1/alpha_(k)|' for i in range(len(Matrix) + 2)]
            return 1/alpha_k_1, df
        
        alpha_k = alpha_k_1
        lambda_k = 1/alpha_k
        x_k = x_k_1
        k+=1

In [72]:
A = np.matrix([
    [2, 1, 1],
    [1, 2.5, 1],
    [1, 1, 3]
], dtype=np.float64)

A = np.matrix([
    [2.2, 1, 0.5, 2],
    [1, 1.3, 2, 1],
    [0.5, 2, 0.5, 1.6],
    [2, 1, 1.6, 2]
], dtype=np.float64)

In [73]:
x0 = sp.Matrix([i+1 for i in range(len(A))])
x0 /= x0.norm()

if (x0.norm() == 1):
    min_lambda, table = inversed_iteration_method(A, x0, epsilon)
    print("Ответ:", min_lambda)

Ответ: 0.222635927132512


In [74]:
print("Эпсилон:", epsilon)
print("Погрешность:", abs(np.linalg.eigvals(A)[np.where(abs(np.linalg.eigvals(A)) == min(abs(np.linalg.eigvals(A))))[0][0]] - min_lambda))
print("Количество итераций:", len(table))

Эпсилон: 1e-13
Погрешность: 1.02945429958368e-13
Количество итераций: 15


In [75]:
table

,X_1,X_2,X_3,X_4,lambda,|1/alpha_(k+1) - 1/alpha_(k)|
0,-1.08922363847045,-0.725666280417149,0.483777520278100,1.56503476245655,0.638963442850532,-0.638963442850532 + sqrt(30)/4
1,-3.14351296527561,-2.65996299886913,0.970246811832855,4.19729701524390,0.238248567201264,0.400714875649268
2,-3.33556950558391,-2.89237941048218,0.983475962580662,4.49497844076047,0.222470477484830,0.0157780897164336
3,-3.32672497767537,-2.89759184708660,0.978837040960016,4.49245126845066,0.222595625471276,0.000125147986445390
4,-3.32528165967419,-2.89777285319211,0.977689177492479,4.49201674427626,0.222617157710777,2.15322395015205e-5
5,-3.32485719092584,-2.89767650917172,0.977539375890251,4.49166394479950,0.222634643261282,1.74855505050420e-5
6,-3.32482041804843,-2.89767348486455,0.977513009987996,4.49164675249031,0.222635495421711,8.52160428860271e-7
7,-3.32481092227954,-2.89767101916582,0.977509869549568,4.49163853622280,0.222635902674604,4.07252893186438e-7
8,-3.32481019761937,-2.89767096927600,0.977509310345649,4.49163823398086,0.222635917655754,1.49811502092589e-8
9,-3.32480999178121,-2.89767091371190,0.977509246383598,4.49163805153028,0.222635926699237,9.04348262942278e-9
